In [1]:
import mediapipe as mp
import numpy as np
import cv2
import time

In [2]:
import torch 
import torch.nn as nn
import torch_geometric
from torch_geometric.data import Data
import torch.optim as optimizer
device = torch.device('cuda:0')
input_size = 1
hidden_size, latent_size = 1, 1
print(device)

cuda:0


In [3]:
if torch.cuda.is_available():
    print("CUDA is available!")
    # CUDA 장치 이름 및 세부 정보 확인
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name}")
else:
    print("CUDA is not available.")

CUDA is available!
Device: NVIDIA GeForce RTX 3060


## 검은 화면에 그래프만 남기기 - 영상 전체

In [22]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=1,
                    enable_segmentation=False,
                    min_detection_confidence=0.5)

# verbose 설정
verbose = False  # 상세 로그 출력 여부
data = "squat"
extension = "mp4"  # 비디오 파일 확장자

# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'data/{data}_processed/{data}_graph_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    
    # Mediapipe Pose 실행
    while cap.isOpened():
        success, image = cap.read()
        if not success or image is None:
            if verbose:
                print(f"End of video {video_file}")
            break
        
        curr_time = time.time()
        
        # BGR 이미지를 RGB로 변환 후 처리
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 검은 화면 생성
        black_frame = np.zeros((h, w, 3), dtype=np.uint8)
        
        # 랜드마크 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
        
        # 출력 영상 저장 (화면 출력은 없음)
        out.write(black_frame)
    
    cap.release()
    out.release()

cv2.destroyAllWindows()


## 검은 화면에 그래프만 남기기 - 영상 초기 15프레임만

In [19]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# verbose 설정
verbose = False  # 상세 로그 출력 여부
data = "not"
extension = "mp4"  # 비디오 파일 확장자

# Pose 모델 초기화 (루프 바깥에서)
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5)

# 비디오 파일 범위 처리 (운동_001.mp4부터 운동_100.mp4까지)
for i in range(1, 101):
    # 비디오 파일 경로 생성
    video_file = f'{data}/{data}_{i:03d}.{extension}'
    cap = cv2.VideoCapture(video_file)
    
    if not cap.isOpened():
        if verbose:
            print(f"Cannot open video {video_file}")
        continue
    
    # 영상 정보 가져오기
    w = round(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # 출력 파일 경로
    out_file = f'data/{data}_start/{data}_graph_start_{i:03d}.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')
    out = cv2.VideoWriter(out_file, fourcc, fps, (w, h))
    
    prev_time = 0
    frame_count = 0  # 프레임 카운트 변수
    
    # Mediapipe Pose 실행
    while cap.isOpened() and frame_count < 15:  # 15프레임까지만 처리
        success, image = cap.read()
        if not success:
            if verbose:
                print(f"End of video {video_file}")
            break
        
        curr_time = time.time()
        
        # 검은 화면 생성
        black_frame = np.zeros((h, w, 3), dtype=np.uint8)
        
        # BGR 이미지를 RGB로 변환 후 처리
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        
        # 랜드마크 그리기
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                black_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2))
    
        # 출력 영상 저장 (화면 출력은 없음)
        out.write(black_frame)
        
        frame_count += 1  # 처리한 프레임 수 증가
    
    cap.release()
    out.release()

# Pose 모델 종료
pose.close()

cv2.destroyAllWindows()


## 웹캠 영상 그래프로 변환

In [ ]:
# Mediapipe Pose 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False, 
                    model_complexity=1, 
                    enable_segmentation=False, 
                    min_detection_confidence=0.5, 
                    min_tracking_confidence=0.5)

# 비디오 입력 설정
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Pose 추론
    results = pose.process(rgb_frame)
    
    # 검은 화면 생성
    black_frame = np.zeros((480, 640, 3), dtype=np.uint8)  # 480x640 해상도, 검은 배경
    
    # Pose 결과를 검은 화면에 그리기
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(
            black_frame, 
            results.pose_landmarks, 
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),  # 랜드마크 스타일
            mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2)  # 연결선 스타일
        )
    
    # 결과 화면 출력
    cv2.imshow('Pose Graph on Black Background', black_frame)
    cv2.imshow('video',frame)
    if cv2.waitKey(1) == ord('q'):  # ESC 키로 종료
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

## 영상에서 노드와 엣지 추출(그래프 데이터로)

In [21]:
data = 'push'

In [22]:
import json
import os
# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 데이터 폴더 설정
output_folder = f"{data}_graph_folder"  # 출력 폴더 (추출된 노드와 엣지를 저장)
# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 비디오 파일 경로 설정 (data_001.mp4부터 data_101.mp4까지 처리)
for i in range(1, 101):  # 1번부터 100번까지 처리
    video_file = f"{data}/{data}_{i:03d}.mov"  # 비디오 파일 경로 생성
    cap = cv2.VideoCapture(video_file)

    if not cap.isOpened():
        print(f"Cannot open video {video_file}")
        continue

    # JSON 파일 생성 (노드 및 엣지 저장)
    json_filename = os.path.join(output_folder, f"{data}_{i:03d}_landmarks_edges.json")
    all_frames_data = []  # 모든 프레임 데이터를 저장할 리스트

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 이미지를 RGB로 변환
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = pose.process(rgb_frame)

        frame_data = {"frame": frame_count, "nodes": [], "edges": []}  # 각 프레임 데이터 초기화

        # Pose landmarks 추출
        if result.pose_landmarks:
            # 각 부위 (노드)의 좌표 저장
            for i, landmark in enumerate(result.pose_landmarks.landmark):
                node_data = {"node_id": i, "x": landmark.x, "y": landmark.y, "z": landmark.z}
                frame_data["nodes"].append(node_data)

            # 엣지 추출: 인체 부위 간 연결
            edges = [
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW),
                (mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW),
                (mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST),
                (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.RIGHT_SHOULDER),
                # 추가적인 연결부위를 추가 가능
            ]

            # 엣지 정보 저장
            for edge in edges:
                start_node = result.pose_landmarks.landmark[edge[0]]
                end_node = result.pose_landmarks.landmark[edge[1]]
                edge_data = {"start_node": edge[0], "end_node": edge[1], 
                             "start_x": start_node.x, "start_y": start_node.y, "start_z": start_node.z,
                             "end_x": end_node.x, "end_y": end_node.y, "end_z": end_node.z}
                frame_data["edges"].append(edge_data)

        # 각 프레임 데이터를 리스트에 추가
        all_frames_data.append(frame_data)

    # JSON 파일에 전체 프레임 데이터 저장
    with open(json_filename, 'w') as json_file:
        json.dump(all_frames_data, json_file, indent=4)

    cap.release()

cv2.destroyAllWindows()
